# Tumor Tracer
## Codigo hecho por Aimé Moral y Gabriel Juan
### El codigo esta separado en dos partes

#### Parte 1: Machine Learning
Tomaremos cada fotos, sin importar el paciente, extraeremos el contorno del cancer y compararemos con la respuesta del dataset

#### Parte 2: Deep Learning
Agruparemos las fotos por paciente, extraeremos los concornos de cada nivel, de forma ordenada. Una vez se tienen todo, se hara una nuve de puntos 3D donde se podra ver el centro del cluster y al ir comparando capa a capa, se podra ver si alguna capa no se ha detectado de una forma coherente con respecto al resto de capas

In [3]:
%pip install numpy opencv-python pandas matplotlib scikit-learn scikit-image scipy seaborn tensorflow keras albumentations

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\UAB\3º 1º\APC\APC_Cass_Kaggle\Tumor-Tracer\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\UAB\3º 1º\APC\APC_Cass_Kaggle\Tumor-Tracer\venv\Scripts\python.exe -m pip install --upgrade pip' command.


  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl (12.9 MB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl (39.0 MB)
  Using cached pandas-2.3.3-cp310-cp310-win_amd64.whl (11.3 MB)
  Using cached matplotlib-3.10.7-cp310-cp310-win_amd64.whl (8.1 MB)
  Using cached scikit_learn-1.7.2-cp310-cp310-win_amd64.whl (8.9 MB)
  Using cached scikit_image-0.25.2-cp310-cp310-win_amd64.whl (12.8 MB)
  Using cached scipy-1.15.3-cp310-cp310-win_amd64.whl (41.3 MB)
  Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
  Using cached tensorflow-2.20.0-cp310-cp310-win_amd64.whl (331.7 MB)
  Using cached keras-3.12.0-py3-none-any.whl (1.5 MB)
  Using cached albumentations-2.0.8-py3-none-any.whl (369 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached pillow-12.0.0-cp310-cp310-win_amd64.whl (7.0 MB)
  Using cached pyparsing-3.2.5-py3-none-any.whl (11

In [1]:
#Librerias

import numpy as np
import cv2
import pandas as pd
import os
import glob
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import jaccard_score
from skimage.filters import roberts, sobel, scharr, prewitt
from scipy import ndimage as nd

In [4]:
import shutil
# Redefinicion de las carpetas
# Rutas de configuración
ORIGINAL_DATASET_PATH = "../data/kaggle_3m/"  # Donde están las carpetas ahora
NEW_FLAT_PATH = "../data/dataset_plano/"      # Donde las quieres poner

# 1. Crear la carpeta nueva si no existe
if not os.path.exists(NEW_FLAT_PATH):
    os.makedirs(NEW_FLAT_PATH)
    print(f"Carpeta creada: {NEW_FLAT_PATH}")

print("--- Iniciando migración de archivos ---")

# 2. Buscar todos los archivos .tif en subcarpetas
# En Windows usamos '**/*.tif' con recursive=True
files = glob.glob(os.path.join(ORIGINAL_DATASET_PATH, '**', '*.tif'), recursive=True)

count = 0
for file_path in files:
    # Obtener solo el nombre del archivo (ej: TCGA_CS_4941_19960909_1.tif)
    filename = os.path.basename(file_path)
    
    # Definir la nueva ruta de destino
    dst_path = os.path.join(NEW_FLAT_PATH, filename)
    
    # Evitar sobrescribir si ya existe
    if not os.path.exists(dst_path):
        # Copiar el archivo (usamos copy2 para preservar metadatos)
        shutil.copy2(file_path, dst_path)
        count += 1

print(f"--- Proceso terminado ---")
print(f"Se han copiado {count} imágenes a '{NEW_FLAT_PATH}'")

--- Iniciando migración de archivos ---
--- Proceso terminado ---
Se han copiado 7858 imágenes a '../data/dataset_plano/'
--- Proceso terminado ---
Se han copiado 7858 imágenes a '../data/dataset_plano/'


## Configuracion y Feature engineering

In [7]:
# Ajusta esta ruta a donde tengas descomprimido tu dataset
DATASET_PATH = "../data/dataset_plano/" 

def extract_features(img):
    """
    Esta función toma una imagen y crea un DataFrame donde cada fila
    es un píxel y cada columna es una característica (feature) extraída
    usando filtros clásicos.
    """
    df = pd.DataFrame()
    
    # Imagen original (1D array)
    img2 = img.reshape(-1)
    df['Original Image'] = img2

    # Feature 1: Gabor Filter (Textura)
    # Generamos varios filtros Gabor con diferentes orientaciones/tamaños
    num = 1
    kernels = []
    for theta in range(2):
        theta = theta / 4. * np.pi
        for sigma in (1, 3):
            lamda = np.pi / 4
            gamma = 0.5
            ksize = 9
            kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, 0, ktype=cv2.CV_32F)
            fimg = cv2.filter2D(img, cv2.CV_8UC3, kernel)
            filtered_img = fimg.reshape(-1)
            df[f'Gabor_{num}'] = filtered_img
            num += 1

    # Feature 2: Canny Edge (Bordes)
    edges = cv2.Canny(img, 100, 200)
    df['Canny Edge'] = edges.reshape(-1)

    # Feature 3: Roberts Edge
    edge_roberts = roberts(img)
    df['Roberts'] = edge_roberts.reshape(-1)

    # Feature 4: Sobel
    edge_sobel = sobel(img)
    df['Sobel'] = edge_sobel.reshape(-1)

    # Feature 5: Gaussian Blur (para suavizado/vecindad)
    gaussian_img = nd.gaussian_filter(img, sigma=3)
    df['Gaussian'] = gaussian_img.reshape(-1)

    return df

def get_contours(mask):
    """
    Obtiene los contornos de una máscara binaria.
    """
    # Asegurarse de que sea uint8
    mask = mask.astype(np.uint8)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def calculate_iou(y_true, y_pred):
    """
    Calcula Intersection over Union (Jaccard Score).
    Es la métrica estándar para segmentación.
    """
    # Aplanar para comparar
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    # Usamos average='binary' asumiendo 0 fondo, 1 tumor
    return jaccard_score(y_true, y_pred, average='binary')

## ENTRENAMIENTO DEL MODELO (Pequeño ejemplo)
Para ML Clásico, no podemos entrenar con TODAS las fotos a la vez en RAM fácilmente. Seleccionamos algunas imágenes representativas para entrenar.

In [8]:
print("--- Buscando imágenes ---")
# Patrón para encontrar imágenes y máscaras
# Buscamos todas las máscaras primero y derivamos las imágenes originales
all_tif_files = glob.glob(os.path.join(DATASET_PATH, '*.tif'))

# Separamos: máscaras terminan en _mask.tif, imágenes NO terminan en _mask.tif
mask_paths = [f for f in all_tif_files if f.endswith('_mask.tif')]
image_paths = [f for f in all_tif_files if not f.endswith('_mask.tif')]

# Filtramos solo los pares donde ambos archivos existen
valid_pairs = [(img, mask) for img, mask in zip(image_paths, mask_paths) if os.path.exists(img)]
image_paths = [p[0] for p in valid_pairs]
mask_paths = [p[1] for p in valid_pairs]

print(f"Encontradas {len(image_paths)} imágenes con sus máscaras correspondientes")

# Tomamos una muestra para entrenar (ej. 100 imágenes) para que corra rápido el ejemplo
# En producción, usa más datos y balancea (mismos pixeles tumor que no-tumor)
subset_size = 50 
train_imgs = image_paths[:subset_size]
train_masks = mask_paths[:subset_size]

print(f"--- Extrayendo características y entrenando Random Forest con {subset_size} imágenes ---")

X_list = []
Y_list = []
print(f"Size of training subset: {len(train_imgs)} images -- Size of masks: {len(train_masks)}")
for img_path, mask_path in zip(train_imgs, train_masks):
    if not os.path.exists(mask_path): continue
    
    # Lectura en escala de grises
    img = cv2.imread(img_path, 0) 
    mask = cv2.imread(mask_path, 0)
    
    # Normalizar máscara a 0 y 1
    mask = mask // 255 

    # Solo entrenamos si la imagen tiene tumor (para que el modelo aprenda qué es un tumor)
    if np.max(mask) == 1:
        features = extract_features(img)
        X_list.append(features)
        Y_list.append(mask.reshape(-1))

if len(X_list) == 0:
    print("Error: No se encontraron máscaras con tumores en el subset.")
    exit()

# Concatenar todos los datos
X = pd.concat(X_list)
Y = np.concatenate(Y_list)

# Entrenar Random Forest
# n_estimators=50 para velocidad, súbelo a 100 o 200 para mejor precisión
model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42, n_jobs=-1)
model.fit(X, Y)

print("--- Modelo Entrenado Exitosamente ---")

--- Buscando imágenes ---
Encontradas 3929 imágenes con sus máscaras correspondientes
--- Extrayendo características y entrenando Random Forest con 50 imágenes ---
Size of training subset: 50 images -- Size of masks: 50
Encontradas 3929 imágenes con sus máscaras correspondientes
--- Extrayendo características y entrenando Random Forest con 50 imágenes ---
Size of training subset: 50 images -- Size of masks: 50
--- Modelo Entrenado Exitosamente ---
--- Modelo Entrenado Exitosamente ---


## Parte 2

In [ ]:
# ==========================================
# PARTE 2: AGRUPACIÓN POR PACIENTE Y ANÁLISIS
# ==========================================

print("--- Iniciando análisis por paciente (Parte 2) ---")

# Diccionario para guardar resultados: { 'Patient_ID': [ {datos_slice_1}, {datos_slice_2} ] }
patients_data = {}

# Usamos algunas imágenes de prueba que el modelo NO haya visto
test_imgs = image_paths[subset_size:subset_size+10] 
test_masks = mask_paths[subset_size:subset_size+10]

for img_path, mask_path in zip(test_imgs, test_masks):
    if not os.path.exists(mask_path): continue

    # 1. Obtener ID del paciente desde la ruta
    # Estructura típica: .../TCGA_CS_4941_19960909/TCGA_CS_4941_19960909_11.tif
    # El ID del paciente suele ser la carpeta padre
    patient_id = os.path.basename(os.path.dirname(img_path))
    filename = os.path.basename(img_path)

    # 2. Cargar imagen y verdad terreno (Ground Truth)
    img = cv2.imread(img_path, 0)
    gt_mask = cv2.imread(mask_path, 0) // 255

    # 3. Predecir con el modelo de ML Clásico
    df_features = extract_features(img)
    prediction_flat = model.predict(df_features)
    prediction_mask = prediction_flat.reshape(img.shape)

    # 4. Limpieza (Morfología matemática)
    # Importante en ML clásico para quitar ruido "pimienta"
    kernel = np.ones((3,3), np.uint8)
    prediction_cleaned = cv2.morphologyEx(prediction_mask.astype(np.uint8), cv2.MORPH_OPEN, kernel)

    # 5. Obtener Contornos
    # Contorno Predicho
    pred_contours = get_contours(prediction_cleaned)
    # Contorno Real (Ground Truth)
    gt_contours = get_contours(gt_mask)

    # 6. Comparación (IoU)
    iou = calculate_iou(gt_mask, prediction_cleaned)

    # 7. Guardar en estructura de datos
    if patient_id not in patients_data:
        patients_data[patient_id] = []

    patients_data[patient_id].append({
        'filename': filename,
        'iou': iou,
        'has_tumor_pred': 1 if np.sum(prediction_cleaned) > 0 else 0,
        'has_tumor_real': 1 if np.sum(gt_mask) > 0 else 0,
        'pred_contours_count': len(pred_contours)
    })

    # Visualización de un ejemplo (Opcional)
    if iou > 0.1 and iou < 0.9: # Mostrar casos interesantes
        plt.figure(figsize=(10, 4))
        
        plt.subplot(1, 3, 1)
        plt.title(f"MRI Original\nPac: {patient_id}")
        plt.imshow(img, cmap='gray')
        
        plt.subplot(1, 3, 2)
        plt.title("Predicción (ML Clásico)")
        plt.imshow(prediction_cleaned, cmap='gray')
        
        plt.subplot(1, 3, 3)
        plt.title(f"Realidad (Ground Truth)\nIoU: {iou:.2f}")
        plt.imshow(gt_mask, cmap='gray')
        
        plt.tight_layout()
        plt.show()
        # Solo mostramos uno y rompemos el bucle de visualización para no llenar la pantalla
        # break 

# ==========================================
# RESUMEN POR PACIENTE
# ==========================================
print("\n--- Resultados Agrupados por Paciente ---")
for pid, slices in patients_data.items():
    avg_iou = np.mean([s['iou'] for s in slices])
    tumor_slices_detected = sum([s['has_tumor_pred'] for s in slices])
    print(f"Paciente: {pid} | Promedio IoU: {avg_iou:.4f} | Slices con tumor detectados: {tumor_slices_detected}")